In [ ]:
# Input is the BRMS model input csv file name
# file_name = "barra2_vpd_phd.csv"
# file_name = "eratos_vpd_phd.csv"
# file_name = "silo_min_temp_vpd_phd.csv"
file_name = "silo_max_temp_vpd_phd.csv"

In [ ]:
import sys

sys.path.append('..')

In [ ]:
import re

# Get the investigated variable from file name (vpd, soil_mois, sdi, dfmc)
pattern = r"(vpd|soil_mois|sdi|dfmc)"
match = re.search(pattern, file_name, re.IGNORECASE)
investigated_variable = match.group(1).lower() if match else None
print("investigated_variable: ", investigated_variable)

# Get remote source and in-situ source from file name
remote_source, insitu_source = file_name.split(f"_{investigated_variable}_")
insitu_source = insitu_source.replace(".csv", "")
print("remote_source:", remote_source)
print("insitu_source:", insitu_source)

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd

# Read the csv into df
df = pd.read_csv(os.path.join('..', 'output', 'csv', file_name))

# Find column names
regex = re.compile(investigated_variable, re.IGNORECASE)
insitu_column = next(
    (col for col in df.columns if regex.search(col) and ~(remote_source in col.lower())), None
)
remote_column = next(
    (col for col in df.columns if regex.search(col) and col != insitu_column), None
)
print("insitu_column: ", insitu_column)
print("remote_column: ", remote_column)

# Plot 6 plots in a figure and save with a clear name
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(9, 9))
fig.suptitle(
    f'Histograms of each variable in the {insitu_source.upper()} dataset mapped to {remote_source.upper()}',
    fontsize=16,
)
axes = axes.flatten()
plot_order = [
    insitu_column,
    remote_column,
    'slope',
    'aspect',
    'relief',
    'veg_cover',
]  # , 'veg_cover']

for ax, col in zip(axes, plot_order):
    ax.hist(df[col], bins=30, color='skyblue', edgecolor='black')
    ax.set_xlabel(col)
    ax.set_ylabel("Frequency")

plt.tight_layout()

# Save fig
plot_dir = os.path.join('..', 'output', 'csv', 'plots')
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
plt.savefig(os.path.join(plot_dir, file_name.replace(".csv", ".png")))

plt.show()